In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import chardet
import os

# ========================
# Función para obtener el texto plano de un link, manejando encoding
# ========================
def get_texto_plano_from_link(link):
    try:
        r = requests.get(link, timeout=10)
        if r.status_code == 200:
            # Intentar detectar el encoding si hay caracteres raros
            enc = r.encoding if r.encoding else 'utf-8'
            try:
                texto_bytes = r.content
                detected_enc = chardet.detect(texto_bytes)['encoding']
                enc = detected_enc if detected_enc else enc
                html = texto_bytes.decode(enc, errors='replace')
            except Exception:
                html = r.text  # Fallback
            soup = BeautifulSoup(html, 'html.parser')
            texto = soup.get_text(separator=' ', strip=True)
        else:
            texto = f"ERROR: Status code {r.status_code}"
    except Exception as e:
        texto = f"ERROR: {e}"
    return texto

# ========================
# Cargar Excel y procesar textos
# ========================
df = pd.read_excel("DataCollected/Super_Excel.xlsx")
df_sample = df  # Tomar solo las primeras 20 filas para test

textos = []
ids = []

# Crear carpeta de resultados si no existe
os.makedirs("Data_Results", exist_ok=True)

for idx, row in df_sample.iterrows():
    link = row['LINK']
    if pd.isna(link) or not str(link).startswith('http'):
        texto = 'ERROR: LINK VACÍO O INVÁLIDO'
    else:
        texto = get_texto_plano_from_link(link)
    textos.append(texto)
    ids.append(row['ID'])
    print(f"[{idx+1}/20] Procesado: {link}")
    time.sleep(0.5)  # Pequeño delay para no sobrecargar el servidor

# Agregar columna al dataframe original
df_sample = df_sample.copy()  # Esto elimina el warning
df_sample['TEXTO_PLANO'] = textos

# Guardar resultado en CSV y Excel

df_sample.to_csv("Data_Results/noticias_texto_20.csv", index=False, encoding='utf-8')
df_sample.to_excel("Data_Results/noticias_texto_20.xlsx", index=False)

print("¡Listo! Archivos guardados como noticias_texto_20.csv y noticias_texto_20.xlsx")


[1/20] Procesado: https://culturagcba.clientes.ejes.com/noticia_completa.cfm?id=19532596
[2/20] Procesado: https://culturagcba.clientes.ejes.com/noticia_completa.cfm?id=19538992
[3/20] Procesado: https://culturagcba.clientes.ejes.com/noticia_completa.cfm?id=19496452
[4/20] Procesado: https://culturagcba.clientes.ejes.com/noticia_completa.cfm?id=19532559
[5/20] Procesado: https://culturagcba.clientes.ejes.com/noticia_completa.cfm?id=19541969
[6/20] Procesado: https://culturagcba.clientes.ejes.com/noticia_completa.cfm?id=19531055
[7/20] Procesado: https://culturagcba.clientes.ejes.com/noticia_completa.cfm?id=19532915
[8/20] Procesado: https://culturagcba.clientes.ejes.com/noticia_completa.cfm?id=19531222
[9/20] Procesado: https://culturagcba.clientes.ejes.com/noticia_completa.cfm?id=19536410
[10/20] Procesado: https://culturagcba.clientes.ejes.com/noticia_completa.cfm?id=19460749
[11/20] Procesado: https://culturagcba.clientes.ejes.com/noticia_completa.cfm?id=19519895
[12/20] Procesado: 

In [11]:
# ========================
# Filtrar filas con texto exitoso (descarta errores, vacíos y noticias no disponibles)
# ========================
frase_no_disponible = "Sintesis de Prensa - Gobierno de la Ciudad de Buenos Aires La noticia solicitada no se encuentra"
frase_no_disponible_2 = "Ministerio de cultura Debes estar logueado"
cond_exito = (
    df_sample["TEXTO_PLANO"].notna() &
    (df_sample["TEXTO_PLANO"].str.strip() != "") &
    (~df_sample["TEXTO_PLANO"].str.startswith("ERROR:")) &
    (~df_sample["TEXTO_PLANO"].str.contains(frase_no_disponible, na=False)) &
    (~df_sample["TEXTO_PLANO"].str.contains(frase_no_disponible_2, na=False)) 
)

df_sample_ok = df_sample[cond_exito].copy()

# Guardar resultado limpio en CSV y Excel
df_sample_ok.to_csv("Data_Results/noticias_texto_30k_limpio.csv", index=False, encoding='utf-8')
df_sample_ok.to_excel("Data_Results/noticias_texto_30k_limpio.xlsx", index=False)

print("¡Archivos limpios guardados como noticias_texto_20_limpio.csv/xlsx. Filas exitosas:", len(df_sample_ok))


¡Archivos limpios guardados como noticias_texto_20_limpio.csv/xlsx. Filas exitosas: 6786
